In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pygad
import wandb

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

tf.random.set_seed(0)
np.random.seed(0)

2023-06-26 12:34:40.810457: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-26 12:34:52.688791: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/macula/SMATousi/.conda/envs/mac-deep/lib/:/home/macula/SMATousi/.conda/envs/mac-deep/lib/python3.7/site-packages/nvidia/cudnn/lib
2023-06-26 12:34:52.690351: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 2433182725021847769
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 10088022016
locality {
  bus_id: 1
  links {
  }
}
incarnation: 4777135397552026585
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:19:00.0, compute capability: 7.5"
xla_global_id: 416903419
, name: "/device:GPU:1"
device_type: "GPU"
memory_limit: 10088022016
locality {
  bus_id: 1
  links {
  }
}
incarnation: 16954091831619390895
physical_device_desc: "device: 1, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:1a:00.0, compute capability: 7.5"
xla_global_id: 2144165316
, name: "/device:GPU:2"
device_type: "GPU"
memory_limit: 10088022016
locality {
  bus_id: 1
  links {
  }
}
incarnation: 2273181530234499188
physical_device_desc: "device: 2, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:67:00.0, compute capability: 7.5"
xla_global_id: 1651660799
, name: "/de

2023-06-26 12:35:07.803353: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-26 12:35:13.902409: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /device:GPU:0 with 9620 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:19:00.0, compute capability: 7.5
2023-06-26 12:35:13.904842: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /device:GPU:1 with 9620 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:1a:00.0, compute capability: 7.5
2023-06-26 12:35:13.905352: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /device:GPU:2 with 9620 MB memory:  -> device: 2, name: NVIDIA GeForce RTX 2080 Ti

# Creating the dataset

In [2]:
num_samples = 10000
num_dimensions = 30

# Generate random samples
raw_data = np.random.rand(num_samples, num_dimensions)

outlier_indices_1 = [0,1,2,3,4,5,6]
outlyin_amount_1 = 20
outlier_indices_2 = [0,2,4,6,8,10,12]
outlyin_amount_2 = 10
outlier_indices_3 = [0,3,6,9,12,15,18]
outlyin_amount_3 = 5
outlier_indices_4 = [0,4,8,12,16,20,24]
outlyin_amount_4 = 2
outlier_indices_5 = [0,5,10,15,20,25,29]
outlyin_amount_5 = 1.1
outlier_indices_6 = [0,1,2,3,4,5,6]
outlyin_amount_6 = [20,15,10,5,2,1.1,1.05]

corrupted_data = raw_data

corrupted_data[100:120, outlier_indices_1] = outlyin_amount_1
corrupted_data[200:220, outlier_indices_2] = outlyin_amount_2
corrupted_data[300:320, outlier_indices_3] = outlyin_amount_3
corrupted_data[400:420, outlier_indices_4] = outlyin_amount_4
corrupted_data[500:520, outlier_indices_5] = outlyin_amount_5
corrupted_data[600:620, outlier_indices_6] = outlyin_amount_6

for outlier_gene_index in range(corrupted_data.shape[1]+1):
    corrupted_data[700+outlier_gene_index,:outlier_gene_index] = outlyin_amount_1
    

# print(raw_data)

In [3]:
corrupted_data[701]

array([20.        ,  0.94229997,  0.21340184,  0.29829028,  0.36824271,
        0.92439828,  0.69054568,  0.08933888,  0.96252338,  0.28198551,
        0.44563463,  0.98380738,  0.42611314,  0.66560582,  0.74637087,
        0.87252321,  0.45021944,  0.09499342,  0.35756956,  0.57396956,
        0.74110312,  0.66910544,  0.57901572,  0.32727592,  0.34571524,
        0.78190065,  0.59662443,  0.57308834,  0.59554356,  0.75843753])

# Training the VAE

In [4]:

class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon


latent_dim = 2

encoder_inputs = keras.Input(shape=(30,))
# x = layers.Conv2D(32, 3, activation="relu", strides=2, padding="same")(encoder_inputs)
# x = layers.Conv2D(64, 3, activation="relu", strides=2, padding="same")(x)
# x = layers.Flatten()(x)
x = layers.Dense(30, activation="tanh")(encoder_inputs)
x = layers.Dense(20, activation="tanh")(x)
x = layers.Dense(18, activation="tanh")(x)
x = layers.Dense(16, activation="tanh")(x)
z_mean = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
z = Sampling()([z_mean, z_log_var])
encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")

latent_inputs = keras.Input(shape=(latent_dim,))
# x = layers.Dense(7 * 7 * 64, activation="relu")(latent_inputs)
# x = layers.Reshape((7, 7, 64))(x)
# x = layers.Conv2DTranspose(64, 3, activation="relu", strides=2, padding="same")(x)
# x = layers.Conv2DTranspose(32, 3, activation="relu", strides=2, padding="same")(x)
# decoder_outputs = layers.Conv2DTranspose(1, 3, activation="sigmoid", padding="same")(x)
x = layers.Dense(16, activation="tanh")(latent_inputs)
x = layers.Dense(18, activation="tanh")(x)
x = layers.Dense(20, activation="tanh")(x)
decoder_outputs = layers.Dense(30, activation="tanh")(x)
decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")



class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super().__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
#             reconstruction_loss = tf.reduce_mean(
#                 tf.reduce_sum(
#                     keras.losses.binary_crossentropy(data, reconstruction), axis=(1, 2)
#                 )
#             )
            reconstruction_loss = tf.keras.losses.MeanSquaredError()(data,reconstruction)
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

creditdata = np.concatenate([corrupted_data], axis=0)
creditdata = np.expand_dims(creditdata, -1).astype("float32")

vae = VAE(encoder, decoder)
vae.compile(optimizer=tf.keras.optimizers.legacy.Adam())
vae.fit(creditdata, epochs=100, batch_size=128)

2023-06-26 12:35:31.812752: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9620 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:19:00.0, compute capability: 7.5
2023-06-26 12:35:31.813263: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 9620 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:1a:00.0, compute capability: 7.5
2023-06-26 12:35:31.813742: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 9620 MB memory:  -> device: 2, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:67:00.0, compute capability: 7.5
2023-06-26 12:35:31.814215: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:3 with 9493 MB memory:  -> device: 3, name: NVIDIA GeForce RTX

Epoch 1/100
79/79 [==============================] - 4s 7ms/step - loss: 1.1542 - reconstruction_loss: 1.1139 - kl_loss: 0.0227
Epoch 2/100
79/79 [==============================] - 1s 6ms/step - loss: 0.8487 - reconstruction_loss: 0.9401 - kl_loss: 0.0014
Epoch 3/100
79/79 [==============================] - 1s 6ms/step - loss: 0.9519 - reconstruction_loss: 0.9364 - kl_loss: 6.6180e-04
Epoch 4/100
79/79 [==============================] - 1s 6ms/step - loss: 1.0967 - reconstruction_loss: 0.9981 - kl_loss: 4.1791e-04
Epoch 5/100
79/79 [==============================] - 1s 6ms/step - loss: 0.8644 - reconstruction_loss: 0.9371 - kl_loss: 3.3279e-04
Epoch 6/100
79/79 [==============================] - 1s 6ms/step - loss: 0.8517 - reconstruction_loss: 0.9368 - kl_loss: 2.4102e-04
Epoch 7/100
79/79 [==============================] - 1s 6ms/step - loss: 0.8884 - reconstruction_loss: 0.9370 - kl_loss: 2.0065e-04
Epoch 8/100
79/79 [==============================] - 1s 6ms/step - loss: 0.9723 - re

79/79 [==============================] - 1s 7ms/step - loss: 0.8976 - reconstruction_loss: 0.9364 - kl_loss: 1.2094e-04
Epoch 63/100
79/79 [==============================] - 1s 6ms/step - loss: 0.9529 - reconstruction_loss: 0.9361 - kl_loss: 7.4132e-05
Epoch 64/100
79/79 [==============================] - 1s 6ms/step - loss: 1.0203 - reconstruction_loss: 0.9361 - kl_loss: 8.2629e-05
Epoch 65/100
79/79 [==============================] - 1s 6ms/step - loss: 1.0159 - reconstruction_loss: 0.9964 - kl_loss: 4.6410e-05
Epoch 66/100
79/79 [==============================] - 1s 7ms/step - loss: 1.0066 - reconstruction_loss: 0.9366 - kl_loss: 6.4475e-05
Epoch 67/100
79/79 [==============================] - 1s 6ms/step - loss: 1.0590 - reconstruction_loss: 0.9365 - kl_loss: 3.7675e-05
Epoch 68/100
79/79 [==============================] - 1s 6ms/step - loss: 0.9333 - reconstruction_loss: 0.9519 - kl_loss: 1.6269e-05
Epoch 69/100
79/79 [==============================] - 1s 7ms/step - loss: 0.9019 -

The error calculation for setting the threshold

In [ ]:
inliers = corrupted_data[5:80,:]
mean_data = np.mean(corrupted_data, axis=0)
outlier1 = corrupted_data[101,:]



# #------------------ replacing the genes here ---------------------

# inliers[:,1] = mean_data[1]

# outlier1[1] = mean_data[1]

# # ----------------------------------------------------------------

ouliers_mean_error = []

for i in range(20):
    
    mean_mean = []
    
    for step in range(10):
        
        outlier1 = corrupted_data[100 + i,:]
        
        outlier1[10] = mean_data[10]

        outlier1 = outlier1.reshape([1,30])

        z_mean, z_log_var, z = vae.encoder(outlier1)
        reconstruction = vae.decoder(z)

        reconstruction_loss = tf.keras.losses.MeanSquaredError()(outlier1,reconstruction)

#         print(reconstruction_loss.numpy())

        mean_mean.append(reconstruction_loss.numpy())

#     print("MEAN_MEAN = ", i, np.mean(np.array(mean_mean)))
#     print("std_MEAN = ", i, np.std(np.array(mean_mean)))
    
    ouliers_mean_error.append(np.mean(np.array(mean_mean)))

print(ouliers_mean_error)
print(np.mean(np.array(ouliers_mean_error)))

In [ ]:
mean_mean = []

for step in range(10):

    mean_ins_error = []

    for index in range(inliers.shape[0]):

        cand = inliers[index,:]

        cand = cand.reshape([1,30])

        mean_data = mean_data.reshape([1,30])

        z_mean, z_log_var, z = vae.encoder(cand)
        reconstruction = vae.decoder(z)

        reconstruction_loss = tf.keras.losses.MeanSquaredError()(cand,reconstruction)

        mean_ins_error.append(reconstruction_loss.numpy())

    #     print("MEAN", index, " = ", reconstruction_loss.numpy())

    mean_ins_error = np.array(mean_ins_error)
    print("mean = ", np.mean(mean_ins_error))
    
    mean_mean.append(np.mean(mean_ins_error))
    

print("MEAN_MEAN = ", np.mean(np.array(mean_mean)))
print("std_MEAN = ", np.std(np.array(mean_mean)))

Setting the threshold and trying to detect the outliers:

In [ ]:
for item_no in range(corrupted_data.shape[0]):
    
    candidate_sample = corrupted_data[item_no,:]
    candidate_sample = candidate_sample.reshape([1,30])
    
    z_mean, z_log_var, z = vae.encoder(candidate_sample)
    reconstruction = vae.decoder(z)

    reconstruction_loss = tf.keras.losses.MeanSquaredError()(candidate_sample,reconstruction)
    
    if reconstruction_loss.numpy() > 1:
        
        print("An Outlier Detected on sample No. ", item_no)

# Genetic Part

In [ ]:
def fitness_func_avg(ga_instance, solution, solution_idx):
    
    inliers = corrupted_data[10:13,:]
    
    avg_ins = np.mean(inliers, axis=0)
    avg_ins = avg_ins.reshape([1,30])
    
    particle = corrupted_data[701,:]
    particle = particle.reshape([1,30])
    
#     abn_subspace = solution * val_features[6728,:]
    
#     abn_subspace = abn_subspace.reshape([1,30])

    avg_in_rec = []
    
    for index in range(inliers.shape[0]):
        
        candidate_inlier = inliers[index,:]
        candidate_inlier = candidate_inlier.reshape([1,30])
        
        in_remain = candidate_inlier * solution
        
        in_normal_subspace = 1 - solution
        
        in_replace = in_normal_subspace * avg_ins
        
        in_candidate = in_remain + in_replace
        
        z_mean, z_log_var, z = vae.encoder(in_candidate)
        in_candidate_rec = vae.decoder(z)
        
        
        rec_loss = tf.keras.losses.MeanSquaredError()(in_candidate,in_candidate_rec)
        
        avg_in_rec.append(rec_loss.numpy())
    
    avg_in_rec = np.array(avg_in_rec)
    avg_in_rec = np.mean(avg_in_rec)
        
        
    
    

#     z_mean, z_log_var, z = vae.encoder(particle)
#     reconstruction_1 = vae.decoder(z)
    
    out_remain = particle * solution
    
    out_normal_subspace = 1 - solution
    
    out_replace = avg_ins * out_normal_subspace
    
    out_candidate = out_remain + out_replace
    
    
    z_mean, z_log_var, z = vae.encoder(out_candidate)
    out_candidate_rec = vae.decoder(z)
    
    rec_loss = tf.keras.losses.MeanSquaredError()(out_candidate,out_candidate_rec)
    rec_loss = rec_loss.numpy()
    
    fitness = rec_loss / avg_in_rec
    
    return fitness


def fitness_func_avg_with_penalty(ga_instance, solution, solution_idx):
    
    inliers = corrupted_data[10:13,:]
    
    avg_ins = np.mean(inliers, axis=0)
    avg_ins = avg_ins.reshape([1,30])
    
    particle = corrupted_data[201,:]
    particle = particle.reshape([1,30])
    
#     abn_subspace = solution * val_features[6728,:]
    
#     abn_subspace = abn_subspace.reshape([1,30])

    avg_in_rec = []
    
    for index in range(inliers.shape[0]):
        
        candidate_inlier = inliers[index,:]
        candidate_inlier = candidate_inlier.reshape([1,30])
        
        in_remain = candidate_inlier * solution
        
        in_normal_subspace = 1 - solution
        
        in_replace = in_normal_subspace * avg_ins
        
        in_candidate = in_remain + in_replace
        
        z_mean, z_log_var, z = vae.encoder(in_candidate)
        in_candidate_rec = vae.decoder(z)
        
        
        rec_loss = tf.keras.losses.MeanSquaredError()(in_candidate,in_candidate_rec)
        
        avg_in_rec.append(rec_loss.numpy())
    
    avg_in_rec = np.array(avg_in_rec)
    avg_in_rec = np.mean(avg_in_rec)
        
        
    
    

#     z_mean, z_log_var, z = vae.encoder(particle)
#     reconstruction_1 = vae.decoder(z)
    
    out_remain = particle * solution
    
    out_normal_subspace = 1 - solution
    
    out_replace = avg_ins * out_normal_subspace
    
    out_candidate = out_remain + out_replace
    
    
    z_mean, z_log_var, z = vae.encoder(out_candidate)
    out_candidate_rec = vae.decoder(z)
    
    rec_loss = tf.keras.losses.MeanSquaredError()(out_candidate,out_candidate_rec)
    rec_loss = rec_loss.numpy()
    
    fitness = rec_loss / avg_in_rec
    
    # adding the penalty here!
    
    num_ones = np.count_nonzero(solution == 1)
    
    penalty_rate = 1.0/corrupted_data.shape[1]
    
    fitness = fitness - num_ones * penalty_rate * fitness
    
    return fitness

def on_generation(ga):
    print("Generation", ga.generations_completed)
    
    solution, solution_fitness, solution_idx = ga_instance.best_solution()
    
    print(solution_fitness)


fitness_function = fitness_func_avg_with_penalty

num_generations = 20
num_parents_mating = 4

sol_per_pop = 100
num_genes = corrupted_data.shape[1]

init_range_low = -2
init_range_high = 5

parent_selection_type = "sss"
keep_parents = 1

space = [[0,1] for i in range(num_genes)]

crossover_type = "single_point"

mutation_type = "random"
mutation_percent_genes = 10

ga_instance = pygad.GA(num_generations=num_generations,
                       num_parents_mating=num_parents_mating,
                       fitness_func=fitness_function,
                       sol_per_pop=sol_per_pop,
                       num_genes=num_genes,
                       init_range_low=init_range_low,
                       init_range_high=init_range_high,
                       parent_selection_type=parent_selection_type,
                       keep_parents=keep_parents,
                       crossover_type=crossover_type,
                       mutation_type=mutation_type,
                       mutation_percent_genes=mutation_percent_genes,
                       on_generation=on_generation,
                       gene_space = space)

In [ ]:
ga_instance.run()

In [ ]:
solution, solution_fitness, solution_idx = ga_instance.best_solution()
print("Parameters of the best solution : {solution}".format(solution=solution))
print("Fitness value of the best solution = {solution_fitness}".format(solution_fitness=solution_fitness))

In [ ]:
ga_instance.plot_fitness()

# For loop for Genetic Algorithms

In [ ]:
best_solutions = []

wandb.init(
    # set the wandb project where this run will be logged
    project="VAE_Outlier_GA_With_Penalty_2",
    
    # track hyperparameters and run metadata
#     config={
#     "learning_rate": 0.02,
#     "architecture": "CNN",
#     "dataset": "CIFAR-100",
#     "epochs": 20,
#     }
)

for out_indexing in range(1,corrupted_data.shape[1]+1):
    
    def fitness_func_avg(ga_instance, solution, solution_idx):

        inliers = corrupted_data[10:13,:]

        avg_ins = np.mean(inliers, axis=0)
        avg_ins = avg_ins.reshape([1,30])

        particle = corrupted_data[700+out_indexing,:]
        particle = particle.reshape([1,30])

    #     abn_subspace = solution * val_features[6728,:]

    #     abn_subspace = abn_subspace.reshape([1,30])

        avg_in_rec = []

        for index in range(inliers.shape[0]):

            candidate_inlier = inliers[index,:]
            candidate_inlier = candidate_inlier.reshape([1,30])

            in_remain = candidate_inlier * solution

            in_normal_subspace = 1 - solution

            in_replace = in_normal_subspace * avg_ins

            in_candidate = in_remain + in_replace

            z_mean, z_log_var, z = vae.encoder(in_candidate)
            in_candidate_rec = vae.decoder(z)


            rec_loss = tf.keras.losses.MeanSquaredError()(in_candidate,in_candidate_rec)

            avg_in_rec.append(rec_loss.numpy())

        avg_in_rec = np.array(avg_in_rec)
        avg_in_rec = np.mean(avg_in_rec)





    #     z_mean, z_log_var, z = vae.encoder(particle)
    #     reconstruction_1 = vae.decoder(z)

        out_remain = particle * solution

        out_normal_subspace = 1 - solution

        out_replace = avg_ins * out_normal_subspace

        out_candidate = out_remain + out_replace


        z_mean, z_log_var, z = vae.encoder(out_candidate)
        out_candidate_rec = vae.decoder(z)

        rec_loss = tf.keras.losses.MeanSquaredError()(out_candidate,out_candidate_rec)
        rec_loss = rec_loss.numpy()

        fitness = rec_loss / avg_in_rec

        return fitness
    
    def fitness_func_avg_with_penalty(ga_instance, solution, solution_idx):
    
        inliers = corrupted_data[10:13,:]

        avg_ins = np.mean(inliers, axis=0)
        avg_ins = avg_ins.reshape([1,30])

        particle = corrupted_data[700+out_indexing,:]
        particle = particle.reshape([1,30])

    #     abn_subspace = solution * val_features[6728,:]

    #     abn_subspace = abn_subspace.reshape([1,30])

        avg_in_rec = []

        for index in range(inliers.shape[0]):

            candidate_inlier = inliers[index,:]
            candidate_inlier = candidate_inlier.reshape([1,30])

            in_remain = candidate_inlier * solution

            in_normal_subspace = 1 - solution

            in_replace = in_normal_subspace * avg_ins

            in_candidate = in_remain + in_replace

            z_mean, z_log_var, z = vae.encoder(in_candidate)
            in_candidate_rec = vae.decoder(z)


            rec_loss = tf.keras.losses.MeanSquaredError()(in_candidate,in_candidate_rec)

            avg_in_rec.append(rec_loss.numpy())

        avg_in_rec = np.array(avg_in_rec)
        avg_in_rec = np.mean(avg_in_rec)





    #     z_mean, z_log_var, z = vae.encoder(particle)
    #     reconstruction_1 = vae.decoder(z)

        out_remain = particle * solution

        out_normal_subspace = 1 - solution

        out_replace = avg_ins * out_normal_subspace

        out_candidate = out_remain + out_replace


        z_mean, z_log_var, z = vae.encoder(out_candidate)
        out_candidate_rec = vae.decoder(z)

        rec_loss = tf.keras.losses.MeanSquaredError()(out_candidate,out_candidate_rec)
        rec_loss = rec_loss.numpy()

        # adding the penalty here!

        num_ones = np.count_nonzero(solution == 1)

        penalty_rate = 1.0 / corrupted_data.shape[1]
        
        regul_param = 10
        
        penalty_rate = penalty_rate / regul_param
        
        fitness = rec_loss / (avg_in_rec + (num_ones * penalty_rate * avg_in_rec))

#         fitness = fitness - num_ones * penalty_rate * fitness

        return fitness

    def on_generation(ga):
        print("Generation", ga.generations_completed)

        solution, solution_fitness, solution_idx = ga_instance.best_solution()
        
        wandb.log({"GA - " + str(out_indexing) + "/solution_fitness": solution_fitness})

        print(solution_fitness)


    fitness_function = fitness_func_avg_with_penalty

    num_generations = 40
    num_parents_mating = 4

    sol_per_pop = 100
    num_genes = corrupted_data.shape[1]

    init_range_low = -2
    init_range_high = 5

    parent_selection_type = "sss"
    keep_parents = 1

    space = [[0,1] for i in range(num_genes)]

    crossover_type = "single_point"

    mutation_type = "random"
    mutation_percent_genes = 15

    ga_instance = pygad.GA(num_generations=num_generations,
                           num_parents_mating=num_parents_mating,
                           fitness_func=fitness_function,
                           sol_per_pop=sol_per_pop,
                           num_genes=num_genes,
                           init_range_low=init_range_low,
                           init_range_high=init_range_high,
                           parent_selection_type=parent_selection_type,
#                            keep_parents=keep_parents,
                           keep_elitism=2,
                           crossover_type=crossover_type,
                           mutation_type=mutation_type,
                           mutation_percent_genes=mutation_percent_genes,
                           on_generation=on_generation,
                           gene_space = space)

    ga_instance.run()

    solution, solution_fitness, solution_idx = ga_instance.best_solution()
    print("##########  End of the ", out_indexing, " epoch ##########")
    print("Parameters of the best solution : {solution}".format(solution=solution))

    best_solutions.append(solution)
    
    sample = corrupted_data[700+out_indexing,:]
    
    TP = 0
    FP = 0
    TN = 0
    FN = 0
    
    for gene_no in range(30):
        
        if ((sample[gene_no] == 20) and (solution[gene_no] == 1)):
            
            TP = TP + 1
        
        elif (sample[gene_no] != 20) and (solution[gene_no] == 1):
            
            FP = FP + 1
            
        elif (sample[gene_no] != 20) and (solution[gene_no] == 0):
            
            TN = TN + 1
            
        elif (sample[gene_no] == 20) and (solution[gene_no] == 0):
            
            FN = FN + 1
    
    precision = TP / (TP + FP)

    recall = TP / (TP + FN)

    F1 = 2 * (precision * recall)/(precision + recall)
    
    MCC = (TP * TN - FP * FN) / np.sqrt((TP + FP) * (TP + FN) * (TN + FP) * (TN + FN))
    
    wandb.log({"Metrics/precision": precision, 
               "Metrics/recall": recall,
               "Metrics/F1": F1,
               "Metrics/MCC": MCC,
               "Metrics/num": out_indexing})
    
best_solutions = np.array(best_solutions)
np.save('best_solutions.npy', best_solutions)

GA - 1/solution_fitness,▁▁▁█▄▅▅▄
GA - 1/solution_fitness,284.54525


Generation 1
276.7299252233733
Generation 2
277.5705507540362
Generation 3
290.9332766875644
Generation 4
294.2255021146463
Generation 5
299.61289183462713
Generation 6
299.61289183462713
Generation 7
299.61289183462713
Generation 8
299.61289183462713
Generation 9
299.61289183462713
Generation 10
299.61289183462713
Generation 11
299.61289183462713
Generation 12
299.61289183462713
Generation 13
299.61289183462713
Generation 14
299.61289183462713
Generation 15
299.61289183462713
Generation 16
299.61289183462713
Generation 17
299.61289183462713
Generation 18
300.3208894847935
Generation 19
300.3208894847935
Generation 20
300.3208894847935
Generation 21
300.3208894847935
Generation 22
300.3208894847935
Generation 23
300.3208894847935
Generation 24
300.63310906609127
Generation 25
300.63310906609127
Generation 26
300.63310906609127
Generation 27
300.63310906609127
Generation 28
300.63310906609127
Generation 29
300.63310906609127
Generation 30
300.63310906609127
Generation 31
300.63310906609

Generation 30
1502.271013910963
Generation 31
1502.271013910963
Generation 32
1502.271013910963
Generation 33
1502.271013910963
Generation 34
1502.271013910963
Generation 35
1502.271013910963
Generation 36
1502.271013910963
Generation 37
1502.271013910963
Generation 38
1502.271013910963
Generation 39
1502.271013910963
Generation 40
1502.271013910963
##########  End of the  6  epoch ##########
Parameters of the best solution : [1. 1. 1. 1. 1. 1. 0. 0. 1. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 0.]
Generation 1
1324.468024341208
Generation 2
1437.3763616309916
Generation 3
1548.8333661505876
Generation 4
1574.6022143030991
Generation 5
1566.020207462406
Generation 6
1679.0928500540963
Generation 7
1666.6287634869486
Generation 8
1660.8597593553545
Generation 9
1660.8597593553545
Generation 10
1660.8597593553545
Generation 11
1660.8597593553545
Generation 12
1660.8597593553545
Generation 13
1660.8597593553545
Generation 14
1660.8597593553545
Generation 15
1660.8597593

2678.739544788582
Generation 12
2678.739544788582
Generation 13
2678.739544788582
Generation 14
2678.739544788582
Generation 15
2678.739544788582
Generation 16
2678.739544788582
Generation 17
2678.739544788582
Generation 18
2678.739544788582
Generation 19
2678.739544788582
Generation 20
2678.739544788582
Generation 21
2678.739544788582
Generation 22
2678.739544788582
Generation 23
2678.739544788582
Generation 24
2678.739544788582
Generation 25
2678.739544788582
Generation 26
2678.739544788582
Generation 27
2678.739544788582
Generation 28
2678.739544788582
Generation 29
2678.739544788582
Generation 30
2678.739544788582
Generation 31
2678.739544788582
Generation 32
2678.739544788582
Generation 33
2678.739544788582
Generation 34
2678.739544788582
Generation 35
2678.739544788582
Generation 36
2678.739544788582
Generation 37
2678.739544788582
Generation 38
2678.739544788582
Generation 39
2678.739544788582
Generation 40
2678.739544788582
##########  End of the  12  epoch ##########
Parameter

Generation 39
3206.6664589428265
Generation 40
3206.6664589428265
##########  End of the  17  epoch ##########
Parameters of the best solution : [0. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 0. 0. 1. 0.]
Generation 1
2806.8321998295264
Generation 2
2942.5034984241183
Generation 3
2951.724418958045
Generation 4
2920.173234683543
Generation 5
3144.185555708955
Generation 6
3290.2731473314902
Generation 7
3198.830635254326
Generation 8
3255.3151736753725
Generation 9
3255.3151736753725


wandb: Network error (ConnectionError), entering retry loop.


In [ ]:
corrupted_data[700+0,:]

# Measurements

F1 calculator

In [ ]:
TP = 3
FP = 1
TN = 22
FN = 4

precision = TP / (TP + FP)

recall = TP / (TP + FN)

F1 = 2 * (precision * recall)/(precision + recall)

MCC = (TP * TN - FP * FN) / np.sqrt((TP + FP) * (TP + FN) * (TN + FP) * (TN + FN))

print("Precision = ", precision)
print("recall = ", recall)
print("F1 = ", F1)
print("MCC = ", MCC)

In [ ]:
for index in range(corrupted_data.shape[1]):
    print("^^^^^^^^^^^^^^^^^^^^^^^^^^^^^")
    print(best_solutions[index,:])
    print(corrupted_data[700+index,:])
    print("######################################3")

In [ ]:
corrupted_data[701,:]

In [ ]:
precs = []
recalls = []
F1s = []
mccs = []


for sample_no in range(1,corrupted_data.shape[1]):
    
    TP = 0
    FP = 0
    TN = 0
    FN = 0
    
    sample = corrupted_data[700+sample_no,:]
    sol = best_solutions[sample_no,:]
    
    for gene_no in range(30):
        
        if ((sample[gene_no] == 20) and (sol[gene_no] == 1)):
            
            TP = TP + 1
        
        elif (sample[gene_no] != 20) and (sol[gene_no] == 1):
            
            FP = FP + 1
            
        elif (sample[gene_no] != 20) and (sol[gene_no] == 0):
            
            TN = TN + 1
            
        elif (sample[gene_no] == 20) and (sol[gene_no] == 0):
            
            FN = FN + 1
    
    

    precision = TP / (TP + FP)

    recall = TP / (TP + FN)

    F1 = 2 * (precision * recall)/(precision + recall)
    
    MCC = (TP * TN - FP * FN) / np.sqrt((TP + FP) * (TP + FN) * (TN + FP) * (TN + FN))

    print("###################################################3")
    print(sol)
    print(sample)
    print("Numbers = ", TP, FP, TN, FN)
    print("Precision = ", precision)
    print("recall = ", recall)
    print("F1 = ", F1)
    print("MCC = ", MCC)
    
    precs.append(precision)
    recalls.append(recall)
    F1s.append(F1)
    mccs.append(MCC)


precs = np.array(precs)
recalls = np.array(recalls)
F1s = np.array(F1s)
mccs = np.array(mccs)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# precs = np.array(precs)
# recalls = np.array(recalls)
# F1s = np.array(F1s)
# mccs = np.array(mccs)

plt.title("Matthews correlation coefficient (MCC)")
plt.plot(mccs)
plt.xlabel("Number of outlying genes")
plt.savefig("MCC.png")

In [ ]:
plt.title("Precision")
plt.plot(precs)
plt.xlabel("Number of outlying genes")
plt.savefig("Precision.png")

In [ ]:
plt.title("Recall")
plt.plot(recalls)
plt.xlabel("Number of outlying genes")
plt.savefig("Recall.png")

In [ ]:
plt.title("F1")
plt.plot(F1s)
plt.xlabel("Number of outlying genes")
plt.savefig("F1.png")